<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/ninologo.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

# Tutorial

This notebook gets you started with using
[Text-Fabric](https://annotation.github.io/text-fabric/) for coding in the Old-Assyrian Letter corpus (cuneiform).

Familiarity with the underlying
[data model](https://annotation.github.io/text-fabric/tf/about/datamodel.html)
is recommended.

## Installing Text-Fabric

### Python

You need to have Python on your system. Most systems have it out of the box,
but alas, that is python2 and we need at least python **3.6**.

Install it from [python.org](https://www.python.org) or from
[Anaconda](https://www.anaconda.com/download).

### TF itself

```
pip3 install text-fabric
```

### Jupyter notebook

You need [Jupyter](http://jupyter.org).

If it is not already installed:

```
pip3 install jupyter
```

## Tip
If you cloned the repository containing this tutorial,
first copy its parent directory to somewhere outside your clone of the repo,
before computing with this it.

If you pull changes from the repository later, it will not conflict with
your computations.

Where you put your tutorial directory is up to you.
It will work from any directory.

## Old Assyrian data

Text-Fabric will fetch the data set for you from the newest github release binaries.

The data will be stored in the `text-fabric-data` in your home directory.

# Features
The data of the corpus is organized in features.
They are *columns* of data.
Think of the corpus as a gigantic spreadsheet, where row 1 corresponds to the
first sign, row 2 to the second sign, and so on, for all 766,000 signs.

The information which reading each sign has, constitutes a column in that spreadsheet.
The Old Assyrian corpus contains over 60 columns, not only for the signs, but also for thousands of other
textual objects, such as clusters, lines, columns, faces, documents.

Instead of putting that information in one big table, the data is organized in separate columns.
We call those columns **features**.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import collections

# Incantation

The simplest way to get going is by this *incantation*:

In [3]:
from tf.app import use

For the very last version, use `hot`.

For the latest release, use `latest`.

If you have cloned the repos (TF app and data), use `clone`.

If you do not want/need to upgrade, leave out the checkout specifiers.

In [4]:
A = use("oldassyrian:clone", checkout="clone", hoist=globals())
# A = use('oldassyrian:hot', checkout="hot", hoist=globals())
# A = use('oldassyrian:latest', checkout="latest", hoist=globals())
# A = use('oldassyrian', hoist=globals())

You can see which features have been loaded, and if you click on a feature name, you find its documentation.
If you hover over a name, you see where the feature is located on your system.

## API

The result of the incantation is that we have a bunch of special variables at our disposal
that give us access to the text and data of the corpus.

At this point it is helpful to throw a quick glance at the text-fabric API documentation
(see the links under **API Members** above).

The most essential thing for now is that we can use `F` to access the data in the features
we've loaded.
But there is more, such as `N`, which helps us to walk over the text, as we see in a minute.

The **API members** above show you exactly which new names have been inserted in your namespace.
If you click on these names, you go to the API documentation for them.

## Search
Text-Fabric contains a flexible search engine, that does not only work for the data,
of this corpus, but also for other corpora and data that you add to corpora.

**Search is the quickest way to come up-to-speed with your data, without too much programming.**

Jump to the dedicated [search](search.ipynb) search tutorial first, to whet your appetite.

The real power of search lies in the fact that it is integrated in a programming environment.
You can use programming to:

* compose dynamic queries
* process query results

Therefore, the rest of this tutorial is still important when you want to tap that power.
If you continue here, you learn all the basics of data-navigation with Text-Fabric.

# Counting

In order to get acquainted with the data, we start with the simple task of counting.

## Count all nodes
We use the
[`N,walk()` generator](https://annotation.github.io/text-fabric/tf/core/nodes.html#tf.core.nodes.Nodes.walk)
to walk through the nodes.

We compared the TF data to a gigantic spreadsheet, where the rows correspond to the signs.
In Text-Fabric, we call the rows `slots`, because they are the textual positions that can be filled with signs.

We also mentioned that there are also other textual objects.
They are the clusters, lines, faces and documents.
They also correspond to rows in the big spreadsheet.

In Text-Fabric we call all these rows *nodes*, and the `N()` generator
carries us through those nodes in the textual order.

Just one extra thing: the `info` statements generate timed messages.
If you use them instead of `print` you'll get a sense of the amount of time that
the various processing steps typically need.

In [5]:
A.indent(reset=True)
A.info("Counting nodes ...")

i = 0
for n in N.walk():
    i += 1

A.info("{} nodes".format(i))

  0.00s Counting nodes ...
  0.19s 1289143 nodes


Here you see it: well over a million nodes.

## What are those nodes?
Every node has a type, like sign, or line, face.
But what exactly are they?

Text-Fabric has two special features, `otype` and `oslots`, that must occur in every Text-Fabric data set.
`otype` tells you for each node its type, and you can ask for the number of `slot`s in the text.

Here we go!

In [6]:
F.otype.slotType

'sign'

In [7]:
F.otype.maxSlot

766501

In [8]:
F.otype.maxNode

1289143

In [9]:
F.otype.all

('document', 'face', 'line', 'word', 'cluster', 'sign')

In [10]:
C.levels.data

(('document', 160.52376963350784, 848587, 853361),
 ('face', 64.35768261964735, 853362, 865271),
 ('line', 6.977061714909885, 865272, 975131),
 ('word', 2.374915608320701, 975132, 1289143),
 ('cluster', 1.7090576841079368, 766502, 848586),
 ('sign', 1, 1, 766501))

This is interesting: above you see all the textual objects, with the average size of their objects,
the node where they start, and the node where they end.

## Count individual object types
This is an intuitive, but inefficient way to count the number of nodes in each type.
Note in passing, how we use the `indent` in conjunction with `info` to produce neat timed
and indented progress messages.

In [11]:
A.indent(reset=True)
A.info("counting objects ...")

for otype in F.otype.all:
    i = 0

    A.indent(level=1, reset=True)

    for n in F.otype.s(otype):
        i += 1

    A.info("{:>7} {}s".format(i, otype))

A.indent(level=0)
A.info("Done")

  0.00s counting objects ...
   |     0.00s    4775 documents
   |     0.00s   11910 faces
   |     0.01s  109860 lines
   |     0.03s  314012 words
   |     0.01s   82085 clusters
   |     0.08s  766501 signs
  0.13s Done


Much more efficient is:

In [12]:
A.indent(reset=True)
A.info("counting objects ...")

for otype in F.otype.all:
    i = 0

    A.indent(level=1, reset=True)
    amount = len(F.otype.s(otype))
    A.info(f"{amount:>7} {otype}s")

A.indent(level=0)
A.info("Done")

  0.00s counting objects ...
   |     0.00s    4775 documents
   |     0.00s   11910 faces
   |     0.00s  109860 lines
   |     0.00s  314012 words
   |     0.00s   82085 clusters
   |     0.00s  766501 signs
  0.00s Done


But nothing beats this:

In [13]:
for lv in C.levels.data:
    print(f"{lv[3] - lv[2] + 1:>6} {lv[0]}s")

  4775 documents
 11910 faces
109860 lines
314012 words
 82085 clusters
766501 signs


# Viewing textual objects

You can use the A API (the extra power) to display cuneiform text.

See the [display](display.ipynb) tutorial.

# Feature statistics

`F`
gives access to all features.
Every feature has a method
`freqList()`
to generate a frequency list of its values, higher frequencies first.
Here are the repeats of numerals (the `-1` comes from a `n(rrr)`:

In [14]:
F.repeat.freqList()

((1, 9533),
 (2, 4139),
 (5, 2975),
 (3, 2448),
 (4, 1787),
 (6, 1236),
 (7, 987),
 (8, 756),
 (9, 395),
 (-1, 2))

Signs have types and clusters have types. We can count them separately:

In [15]:
F.type.freqList("cluster")

(('langalt', 47511),
 ('missing', 28852),
 ('det', 3823),
 ('supplied', 1281),
 ('excised', 349),
 ('uncertain', 269))

In [16]:
F.type.freqList("sign")

(('reading', 691349),
 ('numeral', 30256),
 ('wdiv', 14395),
 ('ellipsis', 12937),
 ('unknown', 10808),
 ('commentline', 6345),
 ('grapheme', 305),
 ('complex', 43),
 ('empty', 30),
 ('comment', 19),
 ('other', 5))

Note in passing that we have nearly 15,000 word dividers!

Finally, the flags:

In [17]:
F.flags.freqList()

(('#', 12950),
 ('?', 2875),
 ('!', 2141),
 ('#?', 684),
 ('#!', 127),
 ('!?', 99),
 ('?!', 37),
 ('#!?', 21),
 ('?#!', 6),
 ('?#', 3),
 ('#?!', 1))

# Word matters

## Top 20 frequent words

We represent words by their essential symbols, collected in the feature *sym* (which also exists for signs).

In [18]:
for (w, amount) in F.sym.freqList("word")[0:20]:
    print(f"{amount:>5} {w}")

12421 a-na
11623 sza
10191 ...
 9683 ku3-babbar
 8560 ma-na
 7809 x
 5849 dumu
 5788 u3
 5717 gin2
 4899 1(disz)
 4360 i-na
 4042 um-ma
 3922 la2
 3839 1(u)
 3726 igi
 2903 u2
 2885 tug2
 2864 2(disz)
 2814 1/2(disz)
 2592 5(disz)


## Word distribution

Let's do a bit more fancy word stuff.

### Hapaxes

A hapax can be found by picking the words with frequency 1

We print 20 hapaxes.

In [19]:
for w in [w for (w, amount) in F.sym.freqList("word") if amount == 1][0:20]:
    print(f'"{w}"')

"&i2-li-esz18-dar"
"...+3(disz)"
"...-a-..."
"...-a-ar"
"...-a-at"
"...-a-hi"
"...-a-ku-nu-ti2"
"...-a-kum"
"...-a-li"
"...-a-na"
"...-a-s,a"
"...-a-szur-ma"
"...-a-ta"
"...-a-tam2"
"...-a-ti2"
"...-a-ti2-szu"
"...-a-wa-tim"
"...-ab"
"...-ab2-ti2-ka3"
"...-ad-ma"


### Small occurrence base

The occurrence base of a word are the documents in which occurs.

We compute the occurrence base of each word.

In [20]:
occurrenceBase = collections.defaultdict(set)

for w in F.otype.s("word"):
    pNum = T.sectionFromNode(w)[0]
    occurrenceBase[F.sym.v(w)].add(pNum)

An overview of how many words have how big occurrence bases:

In [21]:
occurrenceSize = collections.Counter()

for (w, pNums) in occurrenceBase.items():
    occurrenceSize[len(pNums)] += 1

occurrenceSize = sorted(
    occurrenceSize.items(),
    key=lambda x: (-x[1], x[0]),
)

for (size, amount) in occurrenceSize[0:10]:
    print(f"base size {size:>4} : {amount:>5} words")
print("...")
for (size, amount) in occurrenceSize[-10:]:
    print(f"base size {size:>4} : {amount:>5} words")

base size    1 : 19660 words
base size    2 :  3912 words
base size    3 :  1765 words
base size    4 :  1042 words
base size    5 :   650 words
base size    6 :   481 words
base size    7 :   328 words
base size    8 :   279 words
base size    9 :   219 words
base size   10 :   185 words
...
base size 1995 :     1 words
base size 2023 :     1 words
base size 2055 :     1 words
base size 2159 :     1 words
base size 2168 :     1 words
base size 2337 :     1 words
base size 2824 :     1 words
base size 3148 :     1 words
base size 3586 :     1 words
base size 3737 :     1 words


Let's give the predicate *private* to those words whose occurrence base is a single document.

In [22]:
privates = {w for (w, base) in occurrenceBase.items() if len(base) == 1}
len(privates)

19660

### Peculiarity of documents

As a final exercise with words, lets make a list of all documents, and show their

* total number of words
* number of private words
* the percentage of private words: a measure of the peculiarity of the document

In [23]:
docList = []

empty = set()
ordinary = set()

for d in F.otype.s("document"):
    pNum = T.documentName(d)
    words = {F.sym.v(w) for w in L.d(d, otype="word")}
    a = len(words)
    if not a:
        empty.add(pNum)
        continue
    o = len({w for w in words if w in privates})
    if not o:
        ordinary.add(pNum)
        continue
    p = 100 * o / a
    docList.append((pNum, a, o, p))

docList = sorted(docList, key=lambda e: (-e[3], -e[1], e[0]))

print(f"Found {len(empty):>4} empty documents")
print(f"Found {len(ordinary):>4} ordinary documents (i.e. without private words)")

Found   17 empty documents
Found  813 ordinary documents (i.e. without private words)


In [24]:
print(
    "{:<20}{:>5}{:>5}{:>5}\n{}".format(
        "document",
        "#all",
        "#own",
        "%own",
        "-" * 35,
    )
)

for x in docList[0:20]:
    print("{:<20} {:>4} {:>4} {:>4.1f}%".format(*x))
print("...")
for x in docList[-20:]:
    print("{:<20} {:>4} {:>4} {:>4.1f}%".format(*x))

document             #all #own %own
-----------------------------------
P360785                 5    5 100.0%
P358235                 1    1 100.0%
P360454                 1    1 100.0%
P360852                12   11 91.7%
P465862                15   11 73.3%
P360830                19   13 68.4%
P361545                78   52 66.7%
P360142                15   10 66.7%
P357757                78   50 64.1%
P293870                36   21 58.3%
P361474                14    8 57.1%
P360285                 7    4 57.1%
P216609                96   52 54.2%
P359305                13    7 53.8%
P361421                62   33 53.2%
P359433                30   15 50.0%
P368348                16    8 50.0%
P290330                12    6 50.0%
P360453                 4    2 50.0%
P359347                 2    1 50.0%
...
P358110                67    1  1.5%
P358779                67    1  1.5%
P359125                67    1  1.5%
P361502                68    1  1.5%
P359063                70    1  1

# Locality API
We travel upwards and downwards, forwards and backwards through the nodes.
The Locality-API (`L`) provides functions: `u()` for going up, and `d()` for going down,
`n()` for going to next nodes and `p()` for going to previous nodes.

These directions are indirect notions: nodes are just numbers, but by means of the
`oslots` feature they are linked to slots. One node *contains* an other node, if the one is linked to a set of slots that contains the set of slots that the other is linked to.
And one if next or previous to an other, if its slots follow or precede the slots of the other one.

`L.u(node)` **Up** is going to nodes that embed `node`.

`L.d(node)` **Down** is the opposite direction, to those that are contained in `node`.

`L.n(node)` **Next** are the next *adjacent* nodes, i.e. nodes whose first slot comes immediately after the last slot of `node`.

`L.p(node)` **Previous** are the previous *adjacent* nodes, i.e. nodes whose last slot comes immediately before the first slot of `node`.

All these functions yield nodes of all possible otypes.
By passing an optional parameter, you can restrict the results to nodes of that type.

The result are ordered according to the order of things in the text.

The functions return always a tuple, even if there is just one node in the result.

## Going up
We go from the first word to the document it contains.
Note the `[0]` at the end. You expect one document, yet `L` returns a tuple.
To get the only element of that tuple, you need to do that `[0]`.

If you are like me, you keep forgetting it, and that will lead to weird error messages later on.

In [25]:
firstDoc = L.u(1, otype="document")[0]
print(firstDoc)

848587


And let's see all the containing objects of sign 3:

In [26]:
s = 3
for otype in F.otype.all:
    if otype == F.otype.slotType:
        continue
    up = L.u(s, otype=otype)
    upNode = "x" if len(up) == 0 else up[0]
    print("sign {} is contained in {} {}".format(s, otype, upNode))

sign 3 is contained in document 848587
sign 3 is contained in face 853362
sign 3 is contained in line 865272
sign 3 is contained in word 975133
sign 3 is contained in cluster x


## Going next
Let's go to the next nodes of the first document.

In [27]:
afterFirstDoc = L.n(firstDoc)
for n in afterFirstDoc:
    print(
        "{:>7}: {:<13} first slot={:<6}, last slot={:<6}".format(
            n,
            F.otype.v(n),
            E.oslots.s(n)[0],
            E.oslots.s(n)[-1],
        )
    )
secondDoc = L.n(firstDoc, otype="document")[0]

    693: sign          first slot=693   , last slot=693   
 975374: word          first slot=693   , last slot=694   
 865338: line          first slot=693   , last slot=696   
 853365: face          first slot=693   , last slot=720   
 848588: document      first slot=693   , last slot=739   


## Going previous

And let's see what is right before the second document.

In [28]:
for n in L.p(secondDoc):
    print(
        "{:>7}: {:<13} first slot={:<6}, last slot={:<6}".format(
            n,
            F.otype.v(n),
            E.oslots.s(n)[0],
            E.oslots.s(n)[-1],
        )
    )

 848587: document      first slot=1     , last slot=692   
 853364: face          first slot=579   , last slot=692   
 865337: line          first slot=680   , last slot=692   
 975373: word          first slot=692   , last slot=692   
 766521: cluster       first slot=692   , last slot=692   
    692: sign          first slot=692   , last slot=692   


## Going down

We go to the faces of the first document, and just count them.

In [29]:
faces = L.d(firstDoc, otype="face")
print(len(faces))

3


## The first line
We pick two nodes and explore what is above and below them:
the first line and the first word.

In [30]:
for n in [
    F.otype.s("word")[0],
    F.otype.s("line")[0],
]:
    A.indent(level=0)
    A.info("Node {}".format(n), tm=False)
    A.indent(level=1)
    A.info("UP", tm=False)
    A.indent(level=2)
    A.info("\n".join(["{:<15} {}".format(u, F.otype.v(u)) for u in L.u(n)]), tm=False)
    A.indent(level=1)
    A.info("DOWN", tm=False)
    A.indent(level=2)
    A.info("\n".join(["{:<15} {}".format(u, F.otype.v(u)) for u in L.d(n)]), tm=False)
A.indent(level=0)
A.info("Done", tm=False)

Node 975132
   |   UP
   |      |   766502          cluster
   |      |   865272          line
   |      |   853362          face
   |      |   848587          document
   |   DOWN
   |      |   766502          cluster
   |      |   1               sign
Node 865272
   |   UP
   |      |   853362          face
   |      |   848587          document
   |   DOWN
   |      |   975132          word
   |      |   766502          cluster
   |      |   1               sign
   |      |   975133          word
   |      |   2               sign
   |      |   3               sign
   |      |   4               sign
   |      |   975134          word
   |      |   766503          cluster
   |      |   5               sign
Done


# Text API

So far, we have mainly seen nodes and their numbers, and the names of node types.
You would almost forget that we are dealing with text.
So let's try to see some text.

In the same way as `F` gives access to feature data,
`T` gives access to the text.
That is also feature data, but you can tell Text-Fabric which features are specifically
carrying the text, and in return Text-Fabric offers you
a Text API: `T`.

## Formats
Cuneiform text can be represented in a number of ways:

* original ATF, with bracketings and flags
* essential symbols: readings and graphemes, repeats and fractions (of numerals), no flags, no clusterings
* unicode symbols

If you wonder where the information about text formats is stored:
not in the program text-fabric, but in the data set.
It has a feature `otext`, which specifies the formats and which features
must be used to produce them. `otext` is the third special feature in a TF data set,
next to `otype` and `oslots`.
It is an optional feature.
If it is absent, there will be no `T` API.

Here is a list of all available formats in this data set.

In [31]:
sorted(T.formats)

['layout-orig-rich',
 'layout-orig-unicode',
 'text-orig-full',
 'text-orig-plain',
 'text-orig-rich',
 'text-orig-unicode']

## Using the formats

The ` T.text()` function is central to get text representations of nodes. Its most basic usage is

```python
T.text(nodes, fmt=fmt)
```
where `nodes` is a list or iterable of nodes, usually word nodes, and `fmt` is the name of a format.
If you leave out `fmt`, the default `text-orig-full` is chosen.

The result is the text in that format for all nodes specified:

In [32]:
T.text([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], fmt="text-orig-plain")

'lugal lugal-ke-en6 lugala-ki-di2-e re-be-'

There is also another usage of this function:

```python
T.text(node, fmt=fmt)
```

where `node` is a single node.
In this case, the default format is *ntype*`-orig-full` where *ntype* is the type of `node`.

If the format is defined in the corpus, it will be used. Otherwise, the word nodes contained in `node` will be looked up
and represented with the default format `text-orig-full`.

In this way we can sensibly represent a lot of different nodes, such as documents, faces, lines, clusters, words and signs.

We compose a set of example nodes and run `T.text` on them:

In [33]:
exampleNodes = [
    F.otype.s("sign")[0],
    F.otype.s("word")[0],
    F.otype.s("cluster")[0],
    F.otype.s("line")[0],
    F.otype.s("face")[0],
    F.otype.s("document")[0],
]
exampleNodes

[1, 975132, 766502, 865272, 853362, 848587]

In [34]:
for n in exampleNodes:
    print(f"This is {F.otype.v(n)} {n}:")
    print(T.text(n))
    print("")

This is sign 1:
_lugal_ 

This is word 975132:
_lugal_ 

This is cluster 766502:
_lugal_ 

This is line 865272:
_lugal_ lugal-ke-en6 _lugal_

This is face 853362:
_lugal_ lugal-ke-en6 _lugal_a-ki-di2-e re-be-tim _lugal_da-num sza isz-ti2 i-le-ee-ta-wu-ni {d}iszkur da-nu-tam2i-di2-szu-ma isz-tu3 s,i2-itsza-am-szi2-im a-di2 e-ra-ab2sza-am-szi2-im ma-tam2 as,-ba-at-mai-na u4-mi3-im isz-te2-en6a-na 7(u) a-la2-ni ka3-ka3-am a-di2-inru-ba-e-szu-nu u2-s,a-bi4-it u3 a-li-szu-nuu2-ha-li-iq {d}iszkur be-el e-mu-qi2-imu3 esz18-dar be-la2-at ta-ha-zi-imat-ma s,a-bi4-tam2 a-mu-ur-ma li-bi4-tam2a-na na-ri-im a-di2-ma i-nala2-sa3-mi3-a mu-sa3-ri i-bi4-ti2-iq-maza-ar-a-am asz2-ta-ka3-an-ma al-su2-mas,a-bi4-tam2 as,-ba-at li-bi4-tam2i-ma-e u2-sze2-li {d}iszkur u3 esz18-darat#-[ma] _1(disz) li#-im gu4 hi-a_ 6(disz) li-me-e_udu-hi-a_ u2-mi3-sza-ma lu u2-t,a-ba-ah7(disz) li-me-e qa2-ra#-du-a sza i-ra-timu2-mi3-sza-ma ma-ah-ri-a e-ku-lu-ni3(disz) li-me-e la2-si2-mu-u2-asza ar-ka3-tim e-ku-lu-ni_1(disz) li-

## Using the formats
Now let's use those formats to print out the first line in this corpus.

Note that only the formats starting with `text-` are usable for this.

For the `layout-` formats, see [display](display.ipynb).

In [35]:
for fmt in sorted(T.formats):
    if fmt.startswith("text-"):
        print("{}:\n\t{}".format(fmt, T.text(range(1, 12), fmt=fmt)))

text-orig-full:
	_lugal_ lugal-ke-en6 _lugal_a-ki-di2-e re-be-
text-orig-plain:
	lugal lugal-ke-en6 lugala-ki-di2-e re-be-
text-orig-rich:
	lugal lugal-ke-en₆ lugala-ki-di₂-e re-be-
text-orig-unicode:
	𒈗 𒈗𒆠𒅔 𒈗𒀀𒆠𒊹𒂊 𒊑𒁁


If we do not specify a format, the **default** format is used (`text-orig-full`).

In [36]:
T.text(range(1, 12))

'_lugal_ lugal-ke-en6 _lugal_a-ki-di2-e re-be-'

In [37]:
firstLine = F.otype.s("line")[0]
T.text(firstLine)

'_lugal_ lugal-ke-en6 _lugal_'

In [38]:
T.text(firstLine, fmt="text-orig-unicode")

'𒈗 𒈗𒆠𒅔 𒈗'

## Word dividers

First we grab all word dividers in a list.

In [39]:
ds = F.type.s("wdiv")
len(ds)

14395

Then we take the first word divider and look up the line in which it occurs

In [40]:
d = ds[0]
ln = L.u(d, otype="line")[0]
A.webLink(ln)

The ATF source of this line is:

In [41]:
A.getSource(ln)

['6. sza-tu3-uk-ki li-sza-ar-bi4-u2 {d}iszkur / _lugal_']

We use the text formats to display this line in various forms:

In [42]:
T.text(ln)

'sza-tu3-uk-ki li-sza-ar-bi4-u2 {d}iszkur / _lugal_'

In [43]:
T.text(ln, fmt="text-orig-plain")

'sza-tu3-uk-ki li-sza-ar-bi4-u2 d⁼iszkur / lugal'

In [44]:
T.text(ln, fmt="text-orig-rich")

'ša-tu₃-uk-ki li-ša-ar-bi₄-u₂ d⁼iškur / lugal'

In [45]:
T.text(ln, fmt="text-orig-unicode")

'𒊭𒁺𒊌𒆠 𒇷𒊭𒅈𒁁𒌑 𒀭𒅎 𒁹 𒈗'

These characters do not look right, but that is because of the font. We can show the text in the right font with the more advanced functions of Text-Fabric (see also [display](display.ipynb):

In [46]:
A.plain(ln, fmt="text-orig-unicode")

And now with the word divider higlighted:

In [47]:
A.plain(ln, fmt="text-orig-unicode", highlights=set(ds))

The important things to remember are:

* you can supply a list of slot nodes and get them represented in all formats
* you can get non-slot nodes `n` in default format by `T.text(n)`
* you can get non-slot nodes `n` in other formats by `T.text(n, fmt=fmt, descend=True)`

## Whole text in all formats in just 6 seconds
Part of the pleasure of working with computers is that they can crunch massive amounts of data.
The text of the Old Assyrian Letters is a piece of cake.

It takes just ten seconds to have that cake and eat it.
In nearly a dozen formats.

In [48]:
A.indent(reset=True)
A.info("writing plain text of all letters in all text formats")

text = collections.defaultdict(list)

for ln in F.otype.s("line"):
    for fmt in sorted(T.formats):
        if fmt.startswith("text-"):
            text[fmt].append(T.text(ln, fmt=fmt, descend=True))

A.info("done {} formats".format(len(text)))

for fmt in sorted(text):
    print("{}\n{}\n".format(fmt, "\n".join(text[fmt][0:5])))

  0.00s writing plain text of all letters in all text formats
  6.56s done 4 formats
text-orig-full
_lugal_ lugal-ke-en6 _lugal_
a-ki-di2-e re-be-tim _lugal_
da-num sza isz-ti2 i-le-e
e-ta-wu-ni {d}iszkur da-nu-tam2
i-di2-szu-ma isz-tu3 s,i2-it

text-orig-plain
lugal lugal-ke-en6 lugal
a-ki-di2-e re-be-tim lugal
da-num sza isz-ti2 i-le-e
e-ta-wu-ni d⁼iszkur da-nu-tam2
i-di2-szu-ma isz-tu3 s,i2-it

text-orig-rich
lugal lugal-ke-en₆ lugal
a-ki-di₂-e re-be-tim lugal
da-num ša iš-ti₂ i-le-e
e-ta-wu-ni d⁼iškur da-nu-tam₂
i-di₂-šu-ma iš-tu₃ ṣi₂-it

text-orig-unicode
𒈗 𒈗𒆠𒅔 𒈗
𒀀𒆠𒊹𒂊 𒊑𒁁𒁴 𒈗
𒁕𒉏 𒊭 𒅖𒊹 𒄿𒇷𒂊
𒂊𒋫𒉿𒉌 𒀭𒅎 𒁕𒉡𒁮
𒄿𒊹𒋗𒈠 𒅖𒁺 𒍣𒀉



### The full plain text
We write all formats to file, in your `Downloads` folder.

In [49]:
for fmt in T.formats:
    if fmt.startswith("text-"):
        with open(os.path.expanduser(f"~/Downloads/{fmt}.txt"), "w") as f:
            f.write("\n".join(text[fmt]))

## Sections

A section in the letter corpus is a document, a face or a line.
Knowledge of sections is not baked into Text-Fabric.
The config feature `otext.tf` may specify three section levels, and tell
what the corresponding node types and features are.

From that knowledge it can construct mappings from nodes to sections, e.g. from line
nodes to tuples of the form:

    (p-number, face specifier, line number)

You can get the section of a node as a tuple of relevant document, face, and line nodes.
Or you can get it as a passage label, a string.

You can ask for the passage corresponding to the first slot of a node, or the one corresponding to the last slot.

If you are dealing with document and face nodes, you can ask to fill out the line and face parts as well.

Here are examples of getting the section that corresponds to a node and vice versa.

**NB:** `sectionFromNode` always delivers a verse specification, either from the
first slot belonging to that node, or, if `lastSlot`, from the last slot
belonging to that node.

In [50]:
someNodes = (
    F.otype.s("sign")[100000],
    F.otype.s("word")[10000],
    F.otype.s("cluster")[5000],
    F.otype.s("line")[15000],
    F.otype.s("face")[1000],
    F.otype.s("document")[500],
)

In [51]:
for n in someNodes:
    nType = F.otype.v(n)
    d = f"{n:>7} {nType}"
    first = A.sectionStrFromNode(n)
    last = A.sectionStrFromNode(n, lastSlot=True, fillup=True)
    tup = (
        T.sectionTuple(n),
        T.sectionTuple(n, lastSlot=True, fillup=True),
    )
    print(f"{d:<16} - {first:<18} {last:<18} {tup}")

 100001 sign     - P361585 reverse:5  P361585 reverse:5  ((849102, 854752, 879010), (849102, 854752, 879010))
 985132 word     - P360578 reverse:10 P360578 reverse:10 ((848733, 853846, 869271), (848733, 853846, 869271))
 771502 cluster  - P390597 reverse:14 P390597 reverse:14 ((848876, 854199, 873114), (848876, 854199, 873114))
 880272 line     - P358365 left:4     P358365 left:4     ((849149, 854885, 880272), (849149, 854885, 880272))
 854362 face     - P390640 reverse    P390640 reverse:6  ((848941, 854362), (848941, 854362, 874900))
 849087 document - P390603            P390603 obverse:29 ((849087,), (849087, 854715, 878463))


# Clean caches

Text-Fabric pre-computes data for you, so that it can be loaded faster.
If the original data is updated, Text-Fabric detects it, and will recompute that data.

But there are cases, when the algorithms of Text-Fabric have changed, without any changes in the data, that you might
want to clear the cache of precomputed results.

There are two ways to do that:

* Locate the `.tf` directory of your dataset, and remove all `.tfx` files in it.
  This might be a bit awkward to do, because the `.tf` directory is hidden on Unix-like systems.
* Call `TF.clearCache()`, which does exactly the same.

It is not handy to execute the following cell all the time, that's why I have commented it out.
So if you really want to clear the cache, remove the comment sign below.

In [52]:
# TF.clearCache()

# Next steps

By now you have an impression how to compute around in the corpus.
While this is still the beginning, I hope you already sense the power of unlimited programmatic access
to all the bits and bytes in the data set.

Here are a few directions for unleashing that power.

* **[display](display.ipynb)** become an expert in creating pretty displays of your text structures
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **[similarLines](similarLines.ipynb)** spot the similarities between lines

---

See the [cookbook](cookbook) for recipes for small, concrete tasks.

CC-BY Dirk Roorda